In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np

df_air_env = pd.read_csv('../data/air_quality_env_processed.csv')
df_air_idx = pd.read_csv('../data/air_quality_index_processed.csv')
df_humidity = pd.read_csv('../data/averaged_humidity.csv')
df_temperature = pd.read_csv('../data/averaged_temperature.csv')
df_pressure = pd.read_csv('../data/pressure.csv')
df_wind = pd.read_csv('../data/wind.csv')

df_air_env = df_air_env.rename(columns={"report_datetime": "datetime"})
df_air_idx = df_air_idx.rename(columns={"report_datetime": "datetime", "agi": "aqi"})
df_humidity = df_humidity.rename(columns={"Hour": "datetime", "Station": "station", "Relative Humidity(percent)": "humidity"})
df_temperature = df_temperature.rename(columns={"Hour": "datetime", "Station": "station", "Average Max Temp": "max_temp", "Average Min Temp": "min_temp"})
df_pressure = df_pressure.rename(columns={"Automatic Weather Station": "station", "Mean Sea Level Pressure(hPa)": "pressure", "Datetime": "datetime"})
df_wind = df_wind.rename(columns={"Automatic Weather Station": "station", "10-Minute Mean Wind Direction(Compass points)": "wind_direction", "10-Minute Mean Speed(km/hour)": "wind_speed", "10-Minute Maximum Gust(km/hour)": "max_wind_speed", "Datetime": "datetime"})

df_air_env = df_air_env.drop(columns=['Unnamed: 0'])
df_air_idx = df_air_idx.drop(columns=['Unnamed: 0'])

df_air_env['datetime'] = pd.to_datetime(df_air_env['datetime'])
df_air_idx['datetime'] = pd.to_datetime(df_air_idx['datetime'])
df_humidity['datetime'] = pd.to_datetime(df_humidity['datetime'])
df_pressure['datetime'] = pd.to_datetime(df_pressure['datetime'])
df_temperature['datetime'] = pd.to_datetime(df_temperature['datetime'])
df_wind['datetime'] = pd.to_datetime(df_wind['datetime'])


In [3]:
import ast

bearing = {
    "North": 0,
    "Northeast": 45,
    "East": 90,
    "Southeast": 135,
    "South": 180,
    "Southwest": 225,
    "West": 270,
    "Northwest": 315,
}

def format_wind_direction(wind_direction):
    if type(wind_direction) == float:
        wind_direction = None
    elif '[' in wind_direction:
        wind_direction = set(ast.literal_eval(wind_direction.replace(' ', ','))) - set(['Calm', 'Variable', 'nan'])
    else:
        wind_direction = set([wind_direction]) - set(['Calm', 'Variable', 'nan'])

    if wind_direction == set():
        cardinal = -10
    elif wind_direction is None:
        cardinal = None
    else:
        cardinal = sum([bearing[d] for d in wind_direction])

    return cardinal

df_wind['wind_direction'] = df_wind['wind_direction'].apply(format_wind_direction)

In [4]:
# Hong Kong SAR official bounds:
lat_min, lat_max = 22 + 9/60 + 14/3600, 22 + 33/60 + 44/3600    # 22.1333… to 22.5833…
lon_min, lon_max = 113 + 50/60 + 7/3600, 114 + 26/60 + 30/3600  # 113.8167… to 114.5167…

# then build your bins as before:
grid_size = 0.009  # ~1km in degrees
# grid_size = 0.1

lat_bins = np.arange(lat_min, lat_max + grid_size, grid_size)
lon_bins = np.arange(lon_min, lon_max + grid_size, grid_size)
lat_centers = lat_bins[:-1] + grid_size/2
lon_centers = lon_bins[:-1] + grid_size/2

In [5]:
df_epd_stations = pd.read_csv('../data/stations_epd.csv')

In [6]:
import re
df_hko_stations = pd.read_csv('../data/stations_hko.csv')
df_hko_stations["station_code"] = df_hko_stations["station_code"].str.replace(r"[\(\)]", "", regex=True)

dms_re = re.compile(r"""^\s*      # optional leading space
                        (\d+)[°o] # degrees + deg‐symbol (degree sign or "o")
                        (\d+)'    # minutes + apostrophe
                        (\d+)"    # seconds + double‐quote
                     """, re.VERBOSE)

def dms_to_decimal(dms_str):
    m = dms_re.match(dms_str)
    if not m:
        return pd.NA
    deg, mins, secs = map(float, m.groups())
    return deg + mins/60 + secs/3600

df_hko_stations["Latitude"]  = df_hko_stations["Latitude"].apply(dms_to_decimal)
df_hko_stations["Longitude"] = df_hko_stations["Longitude"].apply(dms_to_decimal)
df_hko_stations = df_hko_stations.drop(columns=['Elevation of ground above mean sea-level (metres)', 'Wind', 'Temp', 'Relative Humidity', 'Pressure'])
df_hko_stations

,station,station_code,Latitude,Longitude
0,Beas River,BR1,22.493333,114.105000
1,Bluff Head,BHD,22.197500,114.211944
2,Central Pier,CP1,22.288889,114.155833
3,Cheung Chau,CCH,22.201111,114.026667
4,Clear Water Bay,CWB,22.263333,114.299722
...,...,...,...,...
61,Tap Mun East,TME,22.468333,114.363056
62,Hong Kong Observatory,HKO,22.301944,114.174167
63,Hong Kong International Airport,HKA,22.309444,113.921944
64,Tsing Yi Wind Station,TYW,22.346667,114.086111


In [7]:
def find_cell(lat, lon, lat_bins, lon_bins):
    i = np.digitize(lat,  lat_bins) - 1
    j = np.digitize(lon,  lon_bins) - 1
    if not (0 <= i < len(lat_bins)-1 and 0 <= j < len(lon_bins)-1):
        return pd.Series({'lat_idx':np.nan,'lon_idx':np.nan})
    return pd.Series({'lat_idx':i,'lon_idx':j})

# apply to each station
df_epd_stations[['lat_idx','lon_idx']] = df_epd_stations.apply(
    lambda row: find_cell(row['Latitude'], row['Longitude'], lat_bins, lon_bins),
    axis=1
)

df_hko_stations[['lat_idx','lon_idx']] = df_hko_stations.apply(
    lambda row: find_cell(row['Latitude'], row['Longitude'], lat_bins, lon_bins),
    axis=1
)

df_epd_stations.to_csv('../data/stations_epd_idx.csv', index=False)

In [8]:
df_epd_stations

,station,Latitude,Longitude,station_code,lat_idx,lon_idx
0,CENTRAL/WESTERN,22.284902,114.144417,CWA,14,34
1,KWAI CHUNG,22.357118,114.129594,KCA,22,32
2,KWUN TONG,22.309618,114.231175,KTA,17,43
3,NORTH,22.496698,114.128244,NRA,38,32
4,SOUTHERN,22.247471,114.160132,SA,10,36
5,SHAM SHUI PO,22.330245,114.159135,SSA,19,35
6,SHATIN,22.376278,114.184521,STA,24,38
7,TUNG CHUNG,22.288901,113.943658,TCA,15,12
8,TSEUNG KWAN O,22.317644,114.259563,TKA,18,47
9,TAP MUN,22.471336,114.360722,TMA,35,58


In [9]:
import pandas as pd

# 0) define the global hour\ly index
full_time_index = pd.date_range(
    start="2024-01-01 00:00",
    end  ="2024-12-01 00:00",
    freq ='1H'
)

def temporal_interpolate_fixed(
    df: pd.DataFrame,
    station_col: str,
    time_col: str,
    value_cols: list[str],
    full_idx: pd.DatetimeIndex = full_time_index
) -> pd.DataFrame:
    """
    1) Ensures time_col is datetime
    2) For each station:
       • reindex to full_idx (same for all)
       • interpolate(value_cols) along time
       • reattach station label to every row
    Returns: one row per station per hour in full_idx
    """
    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col])

    out = []
    for stn, grp in df.groupby(station_col, sort=False):
        # set & sort on timestamp
        grp = grp.set_index(time_col).sort_index()

        # reindex to the global index
        grp = grp.reindex(full_idx)

        # interpolate your numeric columns in time
        grp[value_cols] = (
            grp[value_cols]
              .interpolate(method='time', limit_direction='both')
        )

        # put the station name back on every row
        grp[station_col] = stn

        out.append(grp)

    # concatenate, reset index to bring time_col back as a column
    result = (
        pd.concat(out, axis=0)
          .reset_index()
          .rename(columns={'index': time_col})
    )
    return result


df_air_env = temporal_interpolate_fixed(
    df_air_env,
    station_col='station',
    time_col='datetime',
    value_cols=['so2','no','no2','rsp','o3','fsp']
)

df_air_idx = temporal_interpolate_fixed(
    df_air_idx,
    station_col='station',
    time_col='datetime',
    value_cols=['aqi']
)

df_humidity = temporal_interpolate_fixed(
    df_humidity,
    station_col='station',
    time_col='datetime',
    value_cols=['humidity']
)

df_temperature = temporal_interpolate_fixed(
    df_temperature,
    station_col='station',
    time_col='datetime',
    value_cols=['max_temp','min_temp']
)

df_pressure = temporal_interpolate_fixed(
    df_pressure,
    station_col='station',
    time_col='datetime',
    value_cols=['pressure']
)

df_wind = temporal_interpolate_fixed(
    df_wind,
    station_col='station',
    time_col='datetime',
    value_cols=['wind_speed','max_wind_speed', 'wind_direction']
)


In [10]:
manual_map_humidity = {
    'HK Observatory': 'Hong Kong Observatory',
    'HK Park': 'Hong Kong Park',
    'Chek Lap Kok': 'Hong Kong International Airport',
    'Tsing Yi': 'New Tsing Yi Station',
    'Pak Tam Chung': 'Pak Tam Chung (Tsak Yue Wu)',
    'Tai Po': 'Tai Po(Yuen Chau Tsai Park)',
    'Tuen Mun': 'Tuen Mun Children and Juvenile Home',
    }
manual_map_temperature = {
    'HK Observatory': 'Hong Kong Observatory',
    'HK Park': 'Hong Kong Park',
    'Chek Lap Kok': 'Hong Kong International Airport',
    'Tsing Yi': 'New Tsing Yi Station',
    'Pak Tam Chung': 'Pak Tam Chung (Tsak Yue Wu)',
    'Tai Po': 'Tai Po(Yuen Chau Tsai Park)',
    'Tuen Mun': 'Tuen Mun Children and Juvenile Home',
    'Tsuen Wan Ho Koon': 'Tsuen Wan'
}
manual_map_pressure = {
    'HK Observatory': 'Hong Kong Observatory',
    'Chek Lap Kok': 'Hong Kong International Airport',
    'Tai Po': 'Tai Po(Yuen Chau Tsai Park)'
}
manual_map_wind = {
    'Tsing Yi': 'Tsing Yi Wind Station',
    'Tuen Mun': 'Tuen Mun Government Offices',
    'Star Ferry': 'Star Ferry(Kowloon)',
    'Chek Lap Kok': 'Hong Kong International Airport',
    }
others = [df_humidity, df_temperature, df_pressure, df_wind]
for df, mapping in zip(others, [manual_map_humidity,manual_map_temperature,manual_map_pressure,manual_map_wind]):
  df["station"] = (
      df["station"]
        .map(mapping)
        .fillna(df["station"])
  )

df_humidity = pd.merge(df_humidity, df_hko_stations, 'left', left_on='station', right_on='station')
df_temperature = pd.merge(df_temperature, df_hko_stations, 'left', left_on='station', right_on='station')
df_pressure = pd.merge(df_pressure, df_hko_stations, 'left', left_on='station', right_on='station')
df_wind = pd.merge(df_wind, df_hko_stations, 'left', left_on='station', right_on='station')
df_air_env = pd.merge(df_air_env, df_epd_stations, 'left', left_on='station', right_on='station')
df_air_idx = pd.merge(df_air_idx, df_epd_stations, 'left', left_on='station', right_on='station')


df_humidity = df_humidity.drop(columns=['station', 'Latitude', 'Longitude', 'station_code'])
df_temperature = df_temperature.drop(columns=['station', 'Latitude', 'Longitude', 'station_code'])
df_pressure = df_pressure.drop(columns=['station', 'Latitude', 'Longitude', 'station_code'])
df_wind = df_wind.drop(columns=['station', 'Latitude', 'Longitude', 'station_code'])
df_air_env = df_air_env.drop(columns=['station', 'Latitude', 'Longitude', 'station_code'])
df_air_idx = df_air_idx.drop(columns=['station', 'Latitude', 'Longitude', 'station_code'])
df_air_idx = df_air_idx.dropna()


In [11]:
from functools import reduce

dfs = [
    df_air_env,
    df_air_idx,
    df_humidity,
    df_temperature,
    df_pressure,
    df_wind,
]

# Perform outer join using reduce and merge
df_all = reduce(lambda left, right: pd.merge(left, right, on=['datetime','lat_idx','lon_idx'], how='outer'), dfs)
month_to_season = {12:0,1:0,2:0, 3:1,4:1,5:1, 6:2,7:2,8:2, 9:3,10:3,11:3}
df_all['season'] = df_all['datetime'].dt.month.map(month_to_season)
df_all['is_weekend'] = df_all['datetime'].dt.dayofweek.isin([5,6]).astype(int)


In [12]:
features = ['so2', 'no', 'no2', 'rsp', 'o3', 'fsp', 'aqi', 'humidity', 'max_temp', 'min_temp', 'pressure', 'wind_direction', 'wind_speed', 'max_wind_speed', 'season', 'is_weekend']

In [13]:
times    = pd.date_range("2024-01-01 00:00", "2024-12-01 00:00", freq="1H")
H        = len(lat_centers)
W        = len(lon_centers)
lat_idxs = np.arange(H)
lon_idxs = np.arange(W)

full_idx = pd.MultiIndex.from_product(
    [times, lat_idxs, lon_idxs],
    names=["datetime","lat_idx","lon_idx"]
)

In [14]:
ds = df_all.set_index(["datetime","lat_idx","lon_idx"]).to_xarray()

ds = ds.reindex(
    datetime=times,
    lat_idx=lat_idxs,
    lon_idx=lon_idxs
)

images = ds.to_array().transpose("datetime","variable","lat_idx","lon_idx").values
images_filled = images.copy()
T, C, H, W = images.shape

In [15]:
import numpy as np
from scipy.interpolate import griddata
from scipy.spatial import cKDTree

def idw_fill(arr2d, mask, power=2, k=8, eps=1e-12):
    """
    Inverse-distance-weighting fill for NaN regions.

    :param arr2d: 2D array with NaNs where we want to fill
    :param mask: boolean mask of same shape, True where arr2d is NaN
    :param power: distance power parameter p (default 2)
    :param k: number of nearest neighbours to use
    :param eps: small constant to avoid zero-distance blowup
    :return: new array with NaNs replaced by IDW estimates
    """
    H, W = arr2d.shape
    # grid of coordinates
    yy, xx = np.meshgrid(np.arange(H), np.arange(W), indexing='ij')

    # known points
    known_y = yy[~mask]
    known_x = xx[~mask]
    known_vals = arr2d[~mask]
    tree = cKDTree(np.vstack((known_y, known_x)).T)

    # points to fill
    miss_y = yy[mask]
    miss_x = xx[mask]
    # query k nearest
    dists, idxs = tree.query(np.vstack((miss_y, miss_x)).T, k=k, p=2)

    # if k==1, make shapes consistent
    if k == 1:
        dists = dists[:,None]
        idxs  = idxs[:,None]

    # compute weights and weighted sums
    weights = 1.0 / (dists**power + eps)
    weights_sum = weights.sum(axis=1, keepdims=True)
    weights /= weights_sum

    # gather neighbour values
    neighbours = known_vals[idxs]      # shape (n_missing, k)
    filled_vals = np.sum(weights * neighbours, axis=1)

    # build output
    out = arr2d.copy()
    out[mask] = filled_vals
    return out

def fill2d_griddata(arr2d, method='cubic', idw_power=2, idw_k=8):
    """
    Fill 2D array by:
      1) scattered-point interpolation (cubic/linear/nearest) inside hull
      2) IDW extrapolation for points outside hull
    """
    H, W = arr2d.shape
    # 1) meshgrid for interpolation
    yy, xx = np.meshgrid(np.arange(H), np.arange(W), indexing='ij')

    # known data
    mask_known = ~np.isnan(arr2d)
    pts  = np.argwhere(mask_known)                # (N_known, 2)
    vals = arr2d[mask_known]

    # 2) interpolate inside convex hull
    out = griddata(pts, vals, (yy, xx), method=method)

    # 3) find holes (= outside hull for 'cubic'/'linear')
    mask_holes = np.isnan(out)
    if mask_holes.any():
        # IDW‐extrapolate using original arr2d values
        out_idw = idw_fill(out, mask_holes, power=idw_power, k=idw_k)

        # fill only the holes
        out[mask_holes] = out_idw[mask_holes]

    return out

# def fill2d_griddata_nn(arr2d, method='cubic'):
#     X, Y = np.meshgrid(np.arange(H), np.arange(W), indexing='ij')

#     # known points
#     mask = ~np.isnan(arr2d)
#     pts = np.argwhere(mask)
#     vals = arr2d[mask]

#     # interpolate onto full grid
#     # 1) cubic inside hull
#     out_cubic = griddata(pts, vals, (X, Y), method='cubic')

#     # 2) nearest everywhere
#     out_nearest = griddata(pts, vals, (X, Y), method='nearest')

#     # 3) fill holes
#     filled = np.where(np.isnan(out_cubic), out_nearest, out_cubic)

#     return filled



from tqdm.notebook import tqdm
for t in tqdm(range(T)):
    for c in range(C):
        arr = images_filled[t, c]
        images_filled[t, c] = fill2d_griddata(arr, method='cubic', idw_power=2, idw_k=5)
        # images_filled[t, c] = fill2d_griddata_nn(arr, method='cubic')




np.save('images_filled.npy', images_filled)

  0%|          | 0/8041 [00:00<?, ?it/s]